In [1]:
import sys
from pyspark.sql import SparkSession
import re
spark = SparkSession.builder.master("spark://spark-master:7077").appName("SparkByIESB").getOrCreate()

21/11/05 23:39:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
file = spark.sparkContext.textFile("./data/shakespeare.txt")

In [3]:
common_words = ['it', 'a', 'an', 'the', '']
def remove_hamlet(line):
    line_ = re.sub('hamlet@\d*','',line)
    return line_.strip()

def fix_spaces(line):
    line_ = re.sub('\t+',' ',line)
    line_ = re.sub(' +',' ',line_)
    line_ = re.sub('[^\w ]','',line_)
    return line_.strip()

def remove_common_words(line, common_words):
    line_words = [word.lower() for word in line.split(' ') if word not in common_words]
    return line_words

def pre_processing(line):
    line_ = remove_hamlet(line)
    line_ = fix_spaces(line_)
    line_words = remove_common_words(line_, common_words)
    return line_words

In [4]:
file_treated = file.flatMap(lambda line: pre_processing(line))
# file_treated = file_treated.filter(lambda line: line != '')

In [5]:
file_count = file_treated.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a + b).sortBy(lambda x: x[1], ascending=False)

In [6]:
file_count.collect()[:5]

[('and', 172), ('to', 139), ('of', 131), ('in', 85), ('i', 79)]

In [7]:
for (k,v) in file_count.collect():
    print(k,": ",v,sep='')

and: 172
to: 139
of: 131
in: 85
i: 79
you: 78
my: 71
hamlet: 63
not: 59
horatio: 58
that: 53
this: 50
his: 48
with: 47
is: 42
our: 42
lord: 40
your: 40
but: 39
for: 38
as: 34
marcellus: 33
he: 32
the: 32
bernardo: 31
so: 31
do: 27
king: 26
me: 26
be: 26
what: 25
all: 24
have: 23
we: 21
thy: 21
tis: 20
speak: 20
no: 20
a: 20
by: 20
like: 19
most: 19
polonius: 18
father: 18
good: 17
laertes: 17
him: 17
ophelia: 16
now: 15
us: 15
if: 15
which: 15
are: 14
more: 14
hath: 14
will: 14
from: 14
think: 13
let: 13
thou: 13
than: 13
was: 13
had: 13
on: 13
or: 13
at: 12
upon: 12
night: 12
may: 12
these: 12
must: 12
then: 12
it: 12
shall: 12
watch: 11
give: 11
would: 11
know: 11
nor: 11
francisco: 11
thee: 11
well: 11
denmark: 10
them: 10
claudius: 10
heaven: 10
did: 10
some: 10
they: 10
again: 9
take: 9
time: 9
here: 9
nature: 9
queen: 8
enter: 8
heart: 8
farewell: 8
where: 8
very: 8
though: 8
fortinbras: 8
come: 8
much: 8
o: 8
too: 8
act: 7
there: 7
fear: 7
state: 7
why: 7
duty: 7
yet: 7
leave: 7

In [ ]:
#rdd3.saveAsTextFile("./wordocunt/")